In [39]:
import sys
mins = 60

from hmmlearn import hmm

import nilmtk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import warnings
import sys
sys.path.append("../code/")
%matplotlib inline
warnings.filterwarnings("ignore")



In [40]:
from nilmtk import *
import os
import nilmtk
from nilmtk.disaggregate.fhmm_exact import sort_learnt_parameters
from nilmtk.disaggregate.fhmm_exact import create_combined_hmm
from nilmtk.disaggregate.fhmm_exact import FHMM

In [41]:
from dataloader import APPLIANCE_ORDER, get_train_test
num_folds =5
train, test = get_train_test(num_folds=num_folds, fold_num=0)



In [49]:
def learn_hmm_appliance(train_appliance, num_states):
    """
    train_appliance: M homes, D days, 24 hours
    """
    mod = hmm.GaussianHMM(num_states, "full")
    t = train_appliance.copy()
    t = t.reshape(-1, 24, 1)
    t = t[np.random.choice(t.shape[0], 400, replace=False), :, :]
    mod.fit(t)
    return mod


In [50]:
def learn_fhmm(train, num_states):
    models = {}
    for appliance_num, appliance in enumerate(APPLIANCE_ORDER[1:]):
        models[appliance] = learn_hmm_appliance(train[:, 1:, :, :][:, appliance_num, :, :], num_states)
        print(appliance)
        print(models[appliance].means_)
        print(" "*20)
    
    new_learnt_models = OrderedDict()
    for appliance, appliance_model in models.items():
        startprob, means, covars, transmat = sort_learnt_parameters(
                        appliance_model.startprob_, appliance_model.means_,
                        appliance_model.covars_, appliance_model.transmat_)
        new_learnt_models[appliance] = hmm.GaussianHMM(
                    startprob.size, "full", startprob, transmat)
        new_learnt_models[appliance].means_ = means
        new_learnt_models[appliance].covars_ = covars

    learnt_model_combined = create_combined_hmm(new_learnt_models)

    f = FHMM()
    f.model = learnt_model_combined
    f.individual = new_learnt_models
    return f

In [71]:
def make_fhmm_preds(num_states):
    out = []
    for cur_fold in range(5):
        print(cur_fold)
        print("------------")
        train, test = get_train_test(num_folds=num_folds, fold_num=cur_fold)
        f = learn_fhmm(train, num_states)
        print("Training done for fold")
        pred = np.zeros_like(test[:, 1:, :, :])
        for home in range(pred.shape[0]):
            for day in range(pred.shape[2]):
                pred_home_day = f.disaggregate_chunk(pd.Series(test[home, 0, day, :]))[APPLIANCE_ORDER[1:]].values.T
                for appliance_num in range(pred.shape[1]):
                    pred[home, appliance_num, day, :] = pred_home_day[appliance_num, :].flatten()
    
        out.append(pred)
    return np.concatenate(out)



In [74]:
p = make_fhmm_preds(2)

0
------------
hvac
[[ 1803.17747195]
 [  394.07964431]]
                    
fridge
[[  68.12691295]
 [ 127.68550781]]
                    
dr
[[  4.35621739e-01]
 [  6.16498009e+02]]
                    
dw
[[   0.        ]
 [ 129.09060849]]
                    
mw
[[  2.23803475]
 [ 52.37631176]]
                    
Training done for fold
1
------------
hvac
[[  343.23805874]
 [ 1556.70936531]]
                    
fridge
[[  65.40813528]
 [ 119.863694  ]]
                    
dr
[[  6.60995521e-01]
 [  8.29260001e+02]]
                    
dw
[[ 122.48330699]
 [   0.        ]]
                    
mw
[[  2.56481511]
 [ 59.08721007]]
                    
Training done for fold
2
------------
hvac
[[ 1668.63037234]
 [  339.74922618]]
                    
fridge
[[  66.58721777]
 [ 127.41040619]]
                    
dr
[[  4.59269160e-01]
 [  6.22209735e+02]]
                    
dw
[[ 13.71148609]
 [ 13.71148609]]
                    
mw
[[  2.917574 ]
 [ 60.2958219]]
             

In [75]:
p.shape

(68, 5, 112, 24)

In [79]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err ={}
gt = tensor[:, 1:, :, :]


for num_states in range(2, 5):
    pred = make_fhmm_preds(num_states)
    pred = np.minimum(pred, tensor[:, 0:1, :,:])
    err[num_states] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

0
------------
hvac
[[ 1577.26315194]
 [  310.37965629]]
                    
fridge
[[ 112.67828522]
 [  67.14933024]]
                    
dr
[[ 70.74380903]
 [ 70.74380903]]
                    
dw
[[   0.        ]
 [ 124.97880518]]
                    
mw
[[  4.98077452e-177]
 [  1.30364204e+001]]
                    
Training done for fold
1
------------
hvac
[[  248.47510636]
 [ 1383.10667154]]
                    
fridge
[[ 128.28591765]
 [  68.47071913]]
                    
dr
[[ 67.48080027]
 [ 67.48080027]]
                    
dw
[[   0.        ]
 [ 118.78723646]]
                    
mw
[[  2.59725188]
 [ 57.31879101]]
                    
Training done for fold
2
------------
hvac
[[ 1592.402057 ]
 [  310.4831141]]
                    
fridge
[[ 133.85261644]
 [  69.15731133]]
                    
dr
[[  3.52432008e-01]
 [  4.15745467e+02]]
                    
dw
[[ 13.07493055]
 [ 13.07493055]]
                    
mw
[[  2.88647148]
 [ 61.92123984]]
                   

In [80]:
pd.DataFrame(err)

,2,3,4
dr,190.564050,233.896074,268.205929
dw,43.442721,87.228494,104.289350
fridge,35.326861,43.500731,43.736687
hvac,343.392909,386.873645,394.909072
mw,23.589841,39.765363,32.391250
